<a href="https://colab.research.google.com/github/hyunseo-hy/KPI200-Index-Extraction/blob/main/Index_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
<Logic>
Step1. Reading the page source and Prettify
* using urllib.request's urlopen : URL Reading
* using bs4's beautifulsoup : Tag Classify
* source.prettify() : Indentation

Step2. Extract the only data needed
* using the Key word
* Extract for the single data and make function to adjust all data
* Dates : using date_format function
* Prices

Step3. Save as a list
"""

"\n<Logic>\nStep1. Reading the page source and Prettify\n* using urllib.request's urlopen : URL Reading\n* using bs4's beautifulsoup : Tag Classify\n* source.prettify() : Indentation\n\nStep2. Extract the only data needed\n* using the Key word\n* Extract for the single data and make function to adjust all data\n* Dates : using date_format function\n* Prices\n\nStep3. Save as a list\n"

In [4]:
from urllib.request import urlopen
import bs4
import datetime as dt

In [5]:
def date_format(d):
  d = str(d).replace('-','.')
  yyyy = int(d.split('.')[0])
  mm = int(d.split('.')[1])
  dd = int(d.split('.')[2])

  this_date = dt.date(yyyy,mm,dd)
  return this_date

In [26]:
def his_index_naver(index_cd, start_date = "", end_date="", page_n=1, last_page=0):
  if start_date:
    start_date = date_format(start_date)
  else:
    start_date = dt.date.today()

  if end_date:
    end_date = date_format(end_date)
  else:
    end_date = dt.date.today()

  naver_index = "https://finance.naver.com/sise/sise_index_day.naver?code=" + index_cd + "&page=" + str(page_n)
  source = urlopen(naver_index).read()
  source = bs4.BeautifulSoup(source,"lxml")
  dates = source.find_all("td", class_="date")
  prices = source.find_all("td", class_= "number_1")

  for n in range(len(dates)):
    if dates[n].text.split('.')[0].isdigit():
      #Dates Process
      this_date = dates[n].text
      this_date = date_format(this_date)
      if this_date <= end_date and this_date >= start_date:
        #Prices Process
        this_close = prices[n*4].text
        this_close = this_close.replace(',','')
        this_close = float(this_close)
        #Save the dictionary
        historical_prices[this_date] = this_close
      elif this_date < start_date:
        return historical_prices

  if last_page == 0:
    last_page = source.find("td", class_="pgRR").find("a")["href"]
    last_page = last_page.split("&")[1]
    last_page = last_page.split("=")[1]
    last_page = int(last_page)

  #Recall the next page
  if page_n < last_page:
    page_n = page_n+1
    his_index_naver(index_cd, start_date, end_date, page_n, last_page)
  return historical_prices

In [27]:
index_cd = "KPI200"
historical_prices = dict()
his_index_naver(index_cd, "2025-12-21", "2026-1-3")

{datetime.date(2026, 1, 2): 624.17,
 datetime.date(2025, 12, 30): 605.98,
 datetime.date(2025, 12, 29): 605.69,
 datetime.date(2025, 12, 26): 590.08,
 datetime.date(2025, 12, 24): 584.21,
 datetime.date(2025, 12, 23): 584.64,
 datetime.date(2025, 12, 22): 582.73}